In [2]:
import pandas as pd
import pandas_ta as ta

# 1. 数据加载与预处理
df = pd.read_csv("/home/geo/Downloads/geo/_posts/fin/mt4/data/daily/AAPL.csv",skiprows=2)
df.columns = ['Date','Close','High','Low','Open','Volume']

# df = df[df.High != df.Low] # 过滤掉最高价等于最低价的无效数据
# df = df.dropna(subset=['Close']) # 删除收盘价为空的行
# df.reset_index(inplace=True, drop=True)

# 2. 计算基础技术指标
df["EMA"] = ta.ema(df.Close, length=300) # 300周期指数移动平均线
df["ATR"] = ta.atr(df.High, df.Low, df.Close, length=14) # 14周期平均真实波幅

# 3. 计算标准布林带指标 (使用 pandas_ta 库)
# 计算长度 200，标准差 4 的布林带
my_bbands = ta.bbands(df.Close, length=200, std=4)
df = df.join(my_bbands)

# 计算长度 200，标准差 0.5 的布林带
my_bbands = ta.bbands(df.Close, length=200, std=0.5)
df = df.join(my_bbands,rsuffix='_low_std')

# 4. 计算自定义布林带上轨 (BBUP) 的三种变体
# 变体 A: 基于 EMA 的滚动标准差
df['BBUP_EMA'] = df['EMA'] + 4 * df['EMA'].rolling(200).std()

# 变体 B: 基于收盘价的滚动标准差 (以 EMA 为中轴)
df['BBUP_Close'] = df['EMA'] + 4 * df['Close'].rolling(200).std()

# 变体 C: 基于平均价格 (Close, Open, High 的均值) 的滚动标准差
df['BBUP_MidPrice'] = df['EMA'] + 4 * ((df['Close'] + df['Open'] + df['High']) / 3).rolling(200).std()

# 5. 计算退出信号
# 计算下轨：使用 20 周期收盘价标准差的 0.5 倍
df['BBDN'] = df['EMA'] - 0.5 * df['Close'].rolling(20).std()

# 定义退出信号：当收盘价跌破 BBDN 时，标记 EXIT 为 1
df.loc[(df['Close'] < df['BBDN']), 'EXIT'] = 1

# 显示结果
df

,Date,Close,High,Low,Open,Volume,EMA,ATR,BBL_200_2.0_2.0,BBM_200_2.0_2.0,...,BBL_200_2.0_2.0_low_std,BBM_200_2.0_2.0_low_std,BBU_200_2.0_2.0_low_std,BBB_200_2.0_2.0_low_std,BBP_200_2.0_2.0_low_std,BBUP_EMA,BBUP_Close,BBUP_MidPrice,BBDN,EXIT
0,2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-03,71.763725,72.523754,71.539337,71.696167,146322800,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-06,72.335564,72.374169,70.634547,70.885479,118387200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-07,71.995361,72.600968,71.775796,72.345212,108872000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-08,73.153496,73.455095,71.698581,71.698581,132079200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2026-01-23,248.039993,249.410004,244.679993,247.320007,41689000,236.931661,4.871432,175.802191,234.734570,...,175.802191,234.734570,293.666949,50.211930,0.612887,271.235553,355.092190,354.999048,232.291021,NaN
1523,2026-01-26,255.410004,256.559998,249.800003,251.479996,55969200,237.054440,5.132044,176.827998,235.152458,...,176.827998,235.152458,293.476918,49.605656,0.673663,271.749573,353.996081,354.278235,232.601172,NaN
1524,2026-01-27,258.269989,261.950012,258.209991,259.170013,49648300,237.195407,5.232613,177.274606,235.452946,...,177.274606,235.452946,293.631286,49.418231,0.696096,272.276697,353.844075,353.880074,232.999102,NaN
1525,2026-01-28,256.440002,258.859985,254.509995,257.649994,41288000,237.323278,5.169569,177.895469,235.786291,...,177.895469,235.786291,293.677113,49.104485,0.678385,272.784551,353.395467,353.446798,233.446438,NaN


In [4]:
from backtesting import Backtest, Strategy
import pandas as pd
import pandas_ta as ta

# 1. 准备数据
df = pd.read_csv("/home/geo/Downloads/geo/_posts/fin/mt4/data/daily/AAPL.csv",skiprows=2)
df.columns = ['Date','Close','High','Low','Open','Volume']
df.dropna()
print(type(df['Date']))

<class 'pandas.core.series.Series'>


In [ ]:
df['Date'][0]

str

In [ ]:


# 2. 定义策略
class MyStrategy(Strategy):
    def init(self):
        # 使用 self.I 包装指标，确保回测框架能正确绘制它们
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=300)
        # 这里可以加入图片中提到的自定义布林带逻辑
    
    def next(self):
        # 具体的买卖逻辑
        if self.data.Close > self.ema:
            self.buy()
        elif self.data.Close < self.ema:
            self.position.close()

# 3. 运行回测
bt = Backtest(df, MyStrategy, cash=10000, commission=.002)
stats = bt.run()

# 4. 查看结果与绘图
print(stats) # 打印回测统计数据（收益率、夏普比率等）
bt.plot()    # 弹出交互式图表


/home/geo/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/geo/.local/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/tmp/ipykernel_38768/1866094627.py:25: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrategy, cash=10000, commission=.002)
Backtest.run:   0%|          | 0/1227 [00:00<?, ?bar/s]/home/geo/.local/lib/python3.13/site-packages/backtesting/backtesting.py:966: UserWarning: time=302: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/geo/.local/lib/python3.13/site-packages/backtesting/backtesting.py:966: UserWarning: time=303: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/geo/.local/lib/python3.13/site-packages/backtesting/backtesting.py:966: UserWarning: time=304: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/geo/.local/lib/python3.13/site-packages/backtesting/backtesting.py:966: UserWarning: time=305: Broker canceled the relative-sized order due to insufficient margin.
  warnings.wa

Start                                     0.0
End                                    1526.0
Duration                               1526.0
Exposure Time [%]                    58.02227
Equity Final [$]                  10878.24096
Equity Peak [$]                   15012.29276
Commissions [$]                     902.60184
Return [%]                            8.78241
Buy & Hold Return [%]                117.2252
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                           -36.24511
Beta                                  0.38411
Max. Drawdown [%]                   -42.40359
Avg. Drawdown [%]                    -5.43782
Max. Drawdown Duration                 1021.0
Avg. Drawdown Duration               70.82353
# Trades                                 22.0
Win Rate [%]                      

GridPlot(id='p1322', ...)